In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [2]:
EPOCH = 10000
BATCH = 100
PICT = 28
PICT2 = PICT**2
ANS = 10
CHAN = 1
FILTER = 3

In [3]:
# Session確立
sess = tf.InteractiveSession()

In [4]:
# データ読み込み
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
# placeholder
x = tf.placeholder(tf.float32, shape=[None, PICT2])
y_ = tf.placeholder(tf.float32, shape=[None, ANS])

# variable
W = tf.Variable(tf.zeros([PICT2,ANS]))
b = tf.Variable(tf.zeros([ANS]))
#sess.run(tf.global_variables_initializer())

In [6]:
# 画像をreshape
# 4つめの引数はチャンネル数
x_image = tf.reshape(x, [-1,PICT,PICT,CHAN])

In [7]:
# 第一層のweightsとbiasのvariable
#　5*5*1のフィルターを32個
W_conv1 = tf.Variable(tf.truncated_normal([FILTER, FILTER, CHAN, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

# 第一層のconvolutionalとpool
#　1つずつ移動
#　2*2のフィルターを2個ずつ移動
#　出力は14*14*32の画像
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
# 第二層のweightsとbiasのvariable
#　5*5*32のフィルターを64個
W_conv2 = tf.Variable(tf.truncated_normal([FILTER, FILTER, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

# 第二層のconvolutionalとpool
#　1つずつ移動
#　2*2のフィルターを2個ずつ移動
#　出力は7*7*64の画像
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
# Relu層
#　weightsとbiasのvariable
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

#　ベクトルに再変換
#　第一層と第二層でreduceされてできた特徴に対してrelu
#　出力1024のベクトル
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [10]:
# Dropout層
#keep_prob: dropout率のplace_holder
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [11]:
# Softmax層
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [12]:
# 交差エントロピー層
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

In [13]:
# AdamOptimizerで最適化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [14]:
# accuracyを途中確認するための入れ物
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# variable初期化
sess.run(tf.global_variables_initializer())

In [17]:
# summaryの設定 ver1
#tf.summary.scalar('cross_entropy', cross_entropy)
#summary_op = tf.summary.merge_all()
#summary_writer = tf.summary.FileWriter('data', graph=sess.graph)

In [22]:
# summaryの設定 ver2
cost_summary_op_train = tf.summary.scalar("Cross_Entropy_on_Train", cross_entropy)
cost_summary_op_test = tf.summary.scalar("Cross_Entropy_on_Test", cross_entropy)
accuracy_op_train = tf.summary.scalar("Accuracy_on_Train", accuracy)
accuracy_op_test = tf.summary.scalar("Accuracy_on_Test", accuracy)

# summary_op = tf.merge_all_summaries() の代わりに、訓練データとテストデータ用のサマリーを別々にマージする
summary_op_train = tf.summary.merge([cost_summary_op_train, accuracy_op_train])
summary_op_test = tf.summary.merge([cost_summary_op_test, accuracy_op_test])
summary_writer = tf.summary.FileWriter('data', graph=sess.graph)

In [19]:
# accuracyを確認しつつtrain
accracies = []

#10000枚の画像を投げるとキャパオーバーしてしまう（7000枚くらいが限界）
imgtest = mnist.test.images.shape

#実行速度計測
import time
start = time.time()

for i in range(EPOCH):
    batch_x, batch_y = mnist.train.next_batch(BATCH)
    
    if i%100 == 0:
        train_dict = {x:batch_x, y_: batch_y, keep_prob: 1.0}
        train_accuracy = accuracy.eval(feed_dict= train_dict)
        accracies.append(train_accuracy)
        print("step %d, training accuracy %g"%(i, train_accuracy))
        
        #test data をランダムに枚選択
        sed = np.random.permutation(imgtest[0])
        sed = sed[:imgtest/2]
        test_image = mnist.test.images[sed, :]
        test_label = mnist.test.labels[sed, :]
        test_dict = {x:test_image, y_: test_label, keep_prob: 1.0}
        
        #1
        #summary_str = sess.run(summary_op, feed_dict=test_dict)
        #summary_writer.add_summary(summary_str, i)
        
        #2
        summary_str = sess.run(summary_op_train, feed_dict=train_dict)
        summary_writer.add_summary(summary_str, i)
        summary_str = sess.run(summary_op_test, feed_dict=test_dict)
        summary_writer.add_summary(summary_str, i)
        summary_writer.flush()
        
    train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})

elapsed_time = time.time() - start

step 0, training accuracy 0.13
step 100, training accuracy 0.91
step 200, training accuracy 0.94
step 300, training accuracy 0.97
step 400, training accuracy 0.9
step 500, training accuracy 0.96
step 600, training accuracy 0.94
step 700, training accuracy 0.93
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.93
step 1100, training accuracy 0.98
step 1200, training accuracy 0.99
step 1300, training accuracy 0.97
step 1400, training accuracy 0.98
step 1500, training accuracy 0.95
step 1600, training accuracy 0.99
step 1700, training accuracy 0.98
step 1800, training accuracy 0.99
step 1900, training accuracy 0.97
step 2000, training accuracy 0.97
step 2100, training accuracy 0.99
step 2200, training accuracy 1
step 2300, training accuracy 0.98
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 0.97
step 2700, training accuracy 0.99
step 2800, training accuracy 0.99
step 2900, training accuracy 0.99

In [20]:
#実行速度 about 8min
print(("finish_time: {0}".format(int(elapsed_time))) + " [sec]")

finish_time: 253 [sec]


In [21]:
# test_accuracyを表示1　バッチ処理で制御
#batch_tx, batch_ty = mnist.test.next_batch(7000)
#print("test accuracy %g"%accuracy.eval(feed_dict={x:batch_tx, y_: batch_ty, keep_prob: 1.0}))
  #=> 0.9923

test accuracy 0.989143
